In [1]:
!pip install -q --upgrade bitsandbytes accelerate

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
from google.colab import userdata
from huggingface_hub import login
import gradio as gr
import torch

In [3]:
LLAMA = "meta-llama/Llama-3.2-3B-Instruct"

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [9]:
def generate_prices(ticker, market_type, volatility, n_samples, base_price=None):

    system_message = f"""
You are a financial data simulator.
Generate a synthetic OHLCV time series dataset.
"""
    user_message= f"""
Conditions:
- Ticker: {ticker}
- Market type: {market_type}
- Volatility regime: {volatility}
- Number of rows: {n_samples}
- Base Price: {base_price}

Output format:
- Output ONLY CSV.
- Include a header row.
- Columns (in this exact order): timestamp, high, low.
- Use 1-day intervals starting from Day 1 (e.g. "Day 1", "Day 2", ...).
- Ensure: high >= low, low <= high.
"""

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
     ]
    quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
    )
    tokenizer = AutoTokenizer.from_pretrained(LLAMA)
    tokenizer.pad_token = tokenizer.eos_token
    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
    model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)
    outputs = model.generate(inputs, max_new_tokens=2000)
    generated_text = tokenizer.decode(outputs[0],skip_special_tokens=True).strip()

    return generated_text

In [10]:
with gr.Blocks() as demo:
    gr.Markdown("## Synthetic Price Reaction Generator")

    with gr.Row():
        with gr.Column():
            ticker = gr.Textbox(label="Ticker", value="AAPL")
            n_samples = gr.Slider(
                label="Number of samples",
                minimum=7, maximum=30, value=7, step=1
            )
            market_type = gr.Dropdown(
                label="Market",
                choices=["Bullish", "Bearish", "Neutral"],
                value="Neutral"
            )
            volatility = gr.Dropdown(
                label="Volatility Regime",
                choices=["Low", "Medium", "High"],
                value="Medium"
            )
            base_price = gr.Number(label="Base price (optional)", value=100)

            run_btn = gr.Button("Generate")

        with gr.Column():
            generated_box = gr.Textbox(
                label="Output",
                lines=15
            )

    run_btn.click(
        fn=generate_prices,
        inputs=[ticker, market_type, volatility, n_samples, base_price],
        outputs=[generated_box],
    )

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8a32d9dae92dfb7323.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
